In [44]:
import torch
import json
import pandas as pd
from toolz import curry
import numpy as np

from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler

# until I properly package this library, need to allow for relative imports
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from bertron.train_bert import input_dict, evaluate

seed = 666

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [62]:
with open('../data/processed/labels.json', 'r') as f_in:
    label_dict = json.load(f_in)
    
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(label_dict),
    output_attentions=False,
    output_hidden_states=False
)
model.to(device)

# load transfer learning component for Enron corpus
model = model.load_state_dict(torch.load(
    '../bertron/models/finetuned_BERT_epoch_1.model')
)

# hmm, will have to revisit these weights, not currently working
model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

AttributeError: '_IncompatibleKeys' object has no attribute 'eval'

In [20]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

max_len = 512
tokenizer_cur = curry(tokenizer.batch_encode_plus)(
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='max_length', 
    max_length=max_len,
    truncation=True,
    return_tensors='pt'
)

In [34]:
X = pd.read_parquet('../data/processed/X.parquet').sample(frac=.05)

seed = 666
X_train, X_test, y_train, y_test = train_test_split(
    X.index.values, 
    X['label'].values, 
    test_size=0.15, 
    random_state=seed, 
    stratify=X['label'].values
)

X['data_type'] = np.nan * X.shape[0]
X.loc[X_train, 'data_type'] = 'train'
X.loc[X_test, 'data_type'] = 'test'

In [40]:
mask_test = X['data_type']=='test'
encoded_data_test = tokenizer_cur(
    X[mask_test]['body'].values
)

batch_size = 4
input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(X[mask_test]['label'].values)
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

dataloader_test = DataLoader(
    dataset_test, 
    sampler=SequentialSampler(dataset_test), 
    batch_size=batch_size
)

AttributeError: '_IncompatibleKeys' object has no attribute 'eval'

In [ ]:
for batch in dataloader_test:

    batch = tuple(b.to(device) for b in batch)
    inputs = input_dict(batch)

    with torch.no_grad():        
        outputs = model(**inputs)

    loss = outputs[0]
    logits = outputs[1]
    loss_test_total += loss.item()

    logits = logits.detach().cpu().numpy()
    label_ids = inputs['labels'].cpu().numpy()
    predictions.append(logits)
    true_tests.append(label_ids)

loss_test_avg = loss_test_total / len(dataloader_test) 

predictions = np.concatenate(predictions, axis=0)
true_tests = np.concatenate(true_tests, axis=0)

return loss_test_avg, predictions, true_tests

In [47]:
for batch in dataloader_test:
    batch = tuple(b.to(device) for b in batch)
    inputs = input_dict(batch)

    with torch.no_grad():        
        outputs = model(**inputs)

    logits = logits.detach().cpu().numpy()
    label_ids = inputs['labels'].cpu().numpy()
    predictions.append(logits)
    true_tests.append(label_ids)
    print('one')

TypeError: '_IncompatibleKeys' object is not callable